# Physics Informed Neural Networks <br> F1 Car Front Wing Aerodymanics

## PINN

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import csv
from pinn import PINN
import utils

In [ ]:
in_dir = "/Users/ggito/repos/pinns/data/"
points_filename = "front_wing_points_final.csv"
norms_filename = "front_wing_norms_final.csv"

out_dir = "/Users/ggito/repos/pinns/data/"
models_out_dir = "/Users/ggito/repos/pinns/data/models/"
log_filepath = out_dir + "log/log.csv"

In [ ]:
wing_df = pd.read_csv(in_dir + points_filename)
norm_df = pd.read_csv(in_dir + norms_filename)

print(wing_df)
print(norm_df)

In [ ]:
x_max = 1
y_max = 1
z_max = 1
t_max = 1

Nx = 10
Ny = 10
Nz = 10
Nt = 10

dx = x_max / (Nx - 1)
dy = y_max / (Ny - 1)
dz = z_max / (Nz - 1)
dt = t_max / (Nt - 1)

x_test = np.linspace(0, x_max, Nx)
y_test = np.linspace(0, y_max, Ny)
z_test = np.linspace(0, z_max, Nz)
t_test = np.linspace(0, t_max, Nt)

x_grid, y_grid, z_gripd, t_grid = np.meshgrid(x_test, y_test, z_test, t_test)

In [ ]:
input_dim = 4
output_dim = 4
# hidden_units = [32, 32, 32]
# hidden_units = [64, 64, 64, 64]
# hidden_units = [128, 128, 128, 128]
# hidden_units = [256, 256, 256, 256]
# hidden_units = [512, 512]
# hidden_units = [1024, 1024, 1024]
hidden_units = [1024, 1024, 1024, 1024]
# hidden_units = [2048, 2048, 2048, 2048]
# hidden_units = [20, 40, 80, 100, 100, 80, 40, 20]

if torch.backends.mps.is_available():
  device = torch.device("mps")
  x = torch.ones(1, device=device)
  print(x)
else:
  print("MPS device not found.")
  device = "cpu"

# device = "cpu"

pinn = PINN(input_dim, output_dim, hidden_units).to(device)

In [ ]:
# optimizer = torch.optim.Adam(pinn.parameters(), lr=0.001)
optimizer = torch.optim.LBFGS(pinn.parameters())

epochs = 100
Nf = 20000   # num of collocation points -> pde evaluation
N0 = 20000   # num of points to evaluate initial conditons
Nb = 20000   # num of points to evaluate boundary conditions
Nw = 20000   # num of points of the surface of the front wing to evaluate boundary conditions

# Density (rho): 1.2041kg/m^3
# Dynamic viscosity (mu): 1.81e-5 kg/m.s
rho = 1.2
mu = 1.81e-5

# m/s
in_velocity = 20

In [ ]:
def sample_points_in_domain(min, max, num_of_samples):
  return np.random.uniform(min, max, size=num_of_samples)

In [ ]:
def zeros(num):
  return np.zeros(num)

In [ ]:
def ones(num):
  return np.ones(num)

In [ ]:
def create_training_inputs(x_max, y_max, z_max, t_max, Nf, N0, Nb, Nw):
  # TODO: use quasi monte carlo sampling
  # collocation points
  x_f = utils.tensor_from_array(sample_points_in_domain(0, x_max, Nf), device=device, requires_grad=True)
  y_f = utils.tensor_from_array(sample_points_in_domain(0, y_max, Nf), device=device, requires_grad=True)
  z_f = utils.tensor_from_array(sample_points_in_domain(0, z_max, Nf), device=device, requires_grad=True)
  t_f = utils.tensor_from_array(sample_points_in_domain(0, t_max, Nf), device=device, requires_grad=True)
  # xyzt_f = utils.stack_xyzt_tensors(x_f, y_f, z_f, t_f)
  # if stacked in a single tensor, the gradients are not computed correctly

  # initial condition points (t=0)
  x0 = utils.tensor_from_array(sample_points_in_domain(0, x_max, N0), device=device, requires_grad=False)
  y0 = utils.tensor_from_array(sample_points_in_domain(0, y_max, N0), device=device, requires_grad=False)
  z0 = utils.tensor_from_array(sample_points_in_domain(0, z_max, N0), device=device, requires_grad=False)
  t0 = utils.tensor_from_array(zeros(N0), device=device, requires_grad=False)
  xyzt_0 = utils.stack_xyzt_tensors(x0, y0, z0, t0)

  # boundary condition points (inflow, y=1)
  x_b = utils.tensor_from_array(sample_points_in_domain(0, x_max, Nb), device=device, requires_grad=False)
  y_b = utils.tensor_from_array(ones(Nb), device=device, requires_grad=False)
  z_b = utils.tensor_from_array(sample_points_in_domain(0, z_max, Nb), device=device, requires_grad=False)
  t_b = utils.tensor_from_array(sample_points_in_domain(0, t_max, Nb), device=device, requires_grad=False)
  xyzt_b = utils.stack_xyzt_tensors(x_b, y_b, z_b, t_b)

  # points & normal vectors on the surface of the wing
  ## sample Nw wing points with the corresponding normals
  sampled_indices = wing_df.sample(n=Nw).index

  x_w, y_w, z_w = [utils.tensor_from_array(wing_df.loc[sampled_indices, col].values, device=device, requires_grad=False) for col in ['x', 'y', 'z']]
  n_x, n_y, n_z = [utils.tensor_from_array(norm_df.loc[sampled_indices, col].values, device=device, requires_grad=False) for col in ['x', 'y', 'z']]
  t_w = utils.tensor_from_array(sample_points_in_domain(0, t_max, Nw), device=device, requires_grad=False)

  xyzt_w = utils.stack_xyzt_tensors(x_w, y_w, z_w, t_w)
  n_xyz = utils.stack_xyz_tensors(n_x, n_y, n_z)

  return (x_f, y_f, z_f, t_f, xyzt_0, xyzt_b, xyzt_w, n_xyz)

In [ ]:
# torch.save(pinn, models_out_dir + 'pinn10.pt')
checkpoint_epochs = 10

In [ ]:
def closure():

  global epoch_loss

  optimizer.zero_grad()

  training_input = create_training_inputs(x_max, y_max, z_max, t_max, Nf, N0, Nb, Nw)

  total_loss, pde_loss, ic_loss, bc_loss, no_slip_loss, imp_loss = pinn.loss(
                  *training_input,
                  in_velocity,
                  mu, rho,
                  c1=1, c2=1, c3=5, c4=1, c5=1)

  epoch_loss = [total_loss.item(), pde_loss.item(), ic_loss.item(), bc_loss.item(), no_slip_loss.item(), imp_loss.item()]

  total_loss.backward()

  return total_loss

In [ ]:
epoch_loss = [0, 0, 0, 0, 0, 0]

for epoch in range(1, epochs+1):
  optimizer.step(lambda: closure())

  utils.save_log(log_filepath, epoch, epoch_loss)
  utils.print_log(epoch, epoch_loss)

  # TODO
  if np.isnan(epoch_loss[0]):
    print("NaN loss, exiting...")
    break

  if epoch % checkpoint_epochs == 0:
    print("Saving checkpoint...")
    torch.save(pinn, models_out_dir + 'pinn12.pt')


In [ ]:
# torch.save(pinn, model_out_dir + 'pinn10.pt')

In [ ]:
# pinn = torch.load(model_out_dir + 'pinn8.pt')
# pinn.train()
# # loaded_model.eval()